In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
# uploading the dataset
df = pd.read_csv(r"C:\Users\m\Downloads\excel files\vehicles.csv")

In [23]:
df.shape

(426880, 26)

In [27]:
# useless columns
df.drop(['id', 'url', 'region_url', 'county',
         'image_url', 'description', 'posting_date'],
        axis = 1, inplace = True)

In [28]:
# price outliers
df = df[df['price'].between(1000, 100000)]
df.shape

(379910, 19)

In [29]:
# odometer outlier
df = df[df['odometer'] < 10**6]
df.shape

(377389, 19)

In [30]:
# droping year 2022 and null years
df = df[df['year'] != 2022]
df.dropna(subset = 'year', inplace = True)
df.shape

(376246, 19)

In [31]:
# droping cars with no manufacturer and model
df.dropna(subset = ['manufacturer', 'model'], how = 'all', inplace = True)
# filling manufacturer with 'unknown'
df['manufacturer'] = df['manufacturer'].fillna('unknown')
# droping cars with no model
df.dropna(subset = 'model', inplace = True)

df.shape

(371954, 19)

In [32]:
# dropping strange inputs
df = df[df['model'].str.contains('[♿🔥]') == False]
df = df[df['model'].str.contains('^[,-./]') == False]
df = df[df['model'].str.contains('^[\W\d]+$') == False]
df = df[df['model'].str.contains('[@%$*#%*!=]') == False]

df.shape

(359740, 19)

In [33]:
# filling paint_color with 'custom'
df['paint_color'] = df['paint_color'].fillna('custom')

# filling title_status with mode value
df['title_status'] = df['title_status'].fillna('clean')

# filling transmission with mode value
df['transmission'].fillna(df['transmission'].mode().iloc[0], inplace = True)

# filling fuel with mode value
df['fuel'].fillna(df['fuel'].mode().iloc[0], inplace = True)

# filling odometer with the median value
df['odometer'] = df['odometer'].fillna(df['odometer'].median())

# convert VIN to binary
df.loc[df['VIN'].notnull(), 'VIN'] = 1  # replacing exsisting VINs with 1
df['VIN'].fillna(0, inplace = True)  # filling nulls with 0

In [34]:
# populating lat & long with the median value per each 'region'
regions = df['region'].unique()

# Fill missing lat and long based on median values for each region
for region in regions:
    median_lat = df[df['region'] == region]['lat'].median()
    median_long = df[df['region'] == region]['long'].median()
    df.loc[(df['region'] == region) & (df['lat'].isnull()), 'lat'] = median_lat
    df.loc[(df['region'] == region) & (df['long'].isnull()), 'long'] = median_long

In [35]:
# bfill some features
columns = ['cylinders', 'type', 'drive', 'size']

for column in columns:
    df.sort_values(by = ['manufacturer', 'model', column], inplace = True)
    df[column].fillna(method = 'bfill', inplace = True)
    
    print(column, '\n number of missing valus :  ', df[column].isnull().sum() )

cylinders 
 number of missing valus :   15
type 
 number of missing valus :   0
drive 
 number of missing valus :   0
size 
 number of missing valus :   16


In [36]:
# dropping remaining nulls
df.dropna(subset = ['size', 'cylinders'], inplace = True)

In [37]:
(df.isnull().sum()/len(df)) * 100

region           0.000000
price            0.000000
year             0.000000
manufacturer     0.000000
model            0.000000
condition       36.700081
cylinders        0.000000
fuel             0.000000
odometer         0.000000
title_status     0.000000
transmission     0.000000
VIN              0.000000
drive            0.000000
size             0.000000
type             0.000000
paint_color      0.000000
state            0.000000
lat              0.000000
long             0.000000
dtype: float64

In [38]:
df.shape

(359724, 19)

In [39]:
(359724/426880) * 100

84.2681784107946

In [40]:
# Saving the results to a csv file for the next session
df.to_csv('Clean_Used_Cars', index = False)